In [140]:
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [840]:
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x))

def logPredict(X, w): 
    return sigmoid(np.dot(X,w))

def logAvgCost(X, y, w):
    predict = logPredict(X, w)
    cost = y * np.log(predict) + (1.0 - y) * np.log(1.0 - predict)
    cost = - cost 
    return cost.sum() / y.shape[0]

def logAvgGrad(X, y, w):
    predict = logPredict(X, w)
    return np.dot(X.T, predict - y) / y.shape[0]

def updateWeightsGD(X, y, w, lr):
    gradient = logAvgGrad(X, y, w)
    step = lr * gradient 
    return w - gradient 

def trainGD(X, y, w, lr, numIters, logFreq):
    costs = [] 
    for i in range(numIters):
        cost = logAvgCost(X, y, w)
        w = updateWeightsGD(X, y, w, lr)
        if i % logFreq == 0: 
            print("iter:", i, "cost:", cost)
            costs.append(cost)
    return w, costs

def trainGDMulticlass(X, y, lr, numIters, logFreq): 
    uniqueYs = np.unique(y)
    ys = [] 
    for uniqueY in uniqueYs: 
        binaryYs = np.copy(y)
        binaryYs += 1 
        binaryYs[binaryYs != uniqueY + 1] = 0
        binaryYs[binaryYs == uniqueY + 1] = 1
        ys.append(binaryYs)
    ys = np.array(ys)
    ws = np.random.rand(uniqueYs.shape[0],\
        X.shape[1],1)
    costs = []
    for i in range(numIters):
        cost = 0
        for uniqueY in uniqueYs: 
            cost += logAvgCost(X, ys[uniqueY], ws[uniqueY])
            ws[uniqueY] = updateWeightsGD(X, ys[uniqueY],\
                ws[uniqueY], lr)
        cost /= uniqueYs.shape[0]
        if i % logFreq == 0:
            print("iter:", i, "cost:", cost)
            costs.append(cost)
    costs = np.array(costs)
    return ws, costs

def logPredictMulticlass(X, w):
    predictions = [] 
    for i in range(ws.shape[0]):
        predictions.append(logPredict(X,w[i]))
    predictions = np.array(predictions)
    return np.argmax(predictions,axis=0)

def logMulticlassAccuracy(X, w, y):
    predictions = logPredictMulticlass(X, w)
    return np.sum(predictions == y) / y.shape[0]

In [849]:
# Load dataset
digits = load_digits()
digX = digits.images 
digY = digits.target
# Reshape images into 1D vectors 
digX = np.reshape(digX, \
    (digX.shape[0], digX.shape[1] * digX.shape[2]))
# Normalize all images 
for i in range(digX.shape[0]):
    digX[i] = np.divide(digX[i], np.amax(digX,axis=1)[i])

# Split training and test sets 
digXTrain, digXTest, digYTrain, digYTest = \
    train_test_split(digX, digY, test_size=0.20)

# Ensure dimensionality matches 
digYTrain = np.expand_dims(digYTrain, axis=1)
digYTest = np.expand_dims(digYTest, axis=1)

# Stochastic Gradient Descent Training
ws, costs = trainGDMulticlass(digXTrain,\
    digYTrain, 0.01, 10, 1)

# Evaluation 
print(logMulticlassAccuracy(digXTest, ws, digYTest))

iter: 0 cost: 8.768279072857823
iter: 1 cost: 1.027191962525198
iter: 2 cost: 0.43375792370313143
iter: 3 cost: 0.35290558177127307
iter: 4 cost: 0.2990977129983059
iter: 5 cost: 0.2700331623513859
iter: 6 cost: 0.25444014072222493
iter: 7 cost: 0.24308356781952298
iter: 8 cost: 0.2331430435651038
iter: 9 cost: 0.22416311114243084
0.7916666666666666
